In [21]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.datasets import mnist
import cv2
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelBinarizer
from sklearn.metrics import classification_report
##from tensorflow.keras.callbacks import Mode
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow.keras.backend as k
from tensorflow.keras.utils import *
import pickle
import tensorflow as tf
import re


In [22]:
def load_data(no_of_classes):
  (X_train,Y_train),(X_test,Y_test)=tf.keras.datasets.cifar10.load_data()
  X_train=(X_train)/255.0
  X_test=(X_test)/255.0
  Y_train=tf.keras.utils.to_categorical(Y_train,num_classes=no_of_classes)
  Y_test=tf.keras.utils.to_categorical(Y_test,num_classes=no_of_classes)
  return X_train,Y_train,X_test,Y_test

In [16]:
class Simple_CNN_Relu(object):
  def __init__(self,convolution_layers,filters_1_2,filters_3_4,kernel_size,dense_layers,dense_nodes,input_shape,output_shape):
    self.convolution_layers=convolution_layers
    self.filters_1_2=filters_1_2
    self.filters_3_4=filters_3_4
    self.kernel_size=kernel_size
    self.dense_layers=dense_layers
    self.dense_nodes=dense_nodes
    self.input_shape=input_shape
    self.output_shape=output_shape
    self.initialize()

  def initialize(self):

    inputs=tf.keras.Input(shape=(self.input_shape,self.input_shape,3))

    X_in = tf.keras.layers.Conv2D(filters =self.filters_1_2, kernel_size = self.kernel_size, padding = 'same', name = "simple_relu_1",
                                  activation = 'relu')(inputs)

    X_in = tf.keras.layers.Conv2D(filters =self.filters_1_2, kernel_size = self.kernel_size, padding = 'same', name = "simple_relu_2",
                                  activation = 'relu')(X_in)
    
    for i in range(0,self.convolution_layers-2):

      X_in = tf.keras.layers.Conv2D(filters =self.filters_3_4, kernel_size = self.kernel_size, padding = 'same', name = "simple_relu_"+str(i+3),
                                  activation = 'relu')(X_in)

    X_in = tf.keras.layers.Flatten()(X_in)
    
    for i in range(0,self.dense_layers):

      X_in=tf.keras.layers.Dense(units =self.dense_nodes, activation = 'relu', name = "simple_relu_dense"+str(i+1))(X_in)
      
    output_layer=tf.keras.layers.Dense(units =self.output_shape, activation = "softmax", name = "output")(X_in)

    self.model=tf.keras.Model(inputs=inputs,outputs=output_layer,name="simple_relu_cnn")

In [23]:
beta=4
eps=0.1

In [25]:
from keras.utils.generic_utils import get_custom_objects
def soft_relu(x):
    return tf.keras.backend.sigmoid(beta*x)

In [41]:
class VCNN_decoupled_network(object):
  def __init__(self,convolution_layers,filters_1_2,filters_3_4,kernel_size,dense_layers,dense_nodes,input_shape,output_shape):
    self.convolution_layers=convolution_layers
    self.filters_1_2=filters_1_2
    self.filters_3_4=filters_3_4
    self.kernel_size=kernel_size
    self.dense_layers=dense_layers
    self.dense_nodes=dense_nodes
    self.input_shape=input_shape
    self.output_shape=output_shape
    self.initialize()

  def initialize(self):

    inputs=tf.keras.Input(shape=(self.input_shape,self.input_shape,3))

    X_feature = tf.keras.layers.Conv2D(filters =self.filters_1_2, kernel_size = self.kernel_size, padding = 'same', name = "feature_CNN_1",
                                  activation = 'linear')(inputs)
    feature_input=tf.keras.layers.Activation('relu')(X_feature)
    get_custom_objects().update({'custom_activation_soft': tf.keras.layers.Activation(soft_relu)})
    soft_activation=tf.keras.layers.Activation(soft_relu)(X_feature)

    X_value = tf.keras.layers.Conv2D(filters =self.filters_1_2, kernel_size = self.kernel_size, padding = 'same', name = "value_CNN_1",
                                  activation = 'linear')(inputs)
    value_input=tf.keras.layers.Multiply()([X_value,soft_activation])



    X_feature = tf.keras.layers.Conv2D(filters =self.filters_1_2, kernel_size = self.kernel_size, padding = 'same', name = "fetaure_CNN_2",
                                activation = 'linear')(feature_input)
    feature_input=tf.keras.layers.Activation('relu')(X_feature)
    get_custom_objects().update({'custom_activation_soft': tf.keras.layers.Activation(soft_relu)})
    soft_activation=tf.keras.layers.Activation(soft_relu)(X_feature)

    X_value = tf.keras.layers.Conv2D(filters =self.filters_1_2, kernel_size = self.kernel_size, padding = 'same', name = "value_CNN_2",
                                activation = 'linear')(value_input)
    value_input=tf.keras.layers.Multiply()([X_value,soft_activation])
    print(X_value.shape)
    print(soft_activation.shape)
    for i in range(0,self.convolution_layers-2):

      X_feature = tf.keras.layers.Conv2D(filters =self.filters_3_4, kernel_size = self.kernel_size, padding = 'same', name = "fetaure_CNN_"+str(i+3),
                                  activation = 'linear')(feature_input)
      feature_input=tf.keras.layers.Activation('relu')(X_feature)
      get_custom_objects().update({'custom_activation_soft': tf.keras.layers.Activation(soft_relu)})
      soft_activation=tf.keras.layers.Activation(soft_relu)(X_feature)

      X_value = tf.keras.layers.Conv2D(filters =self.filters_3_4, kernel_size = self.kernel_size, padding = 'same', name = "value_CNN_"+str(i+3),
                                  activation = 'linear')(value_input)
      value_input=tf.keras.layers.Multiply()([X_value,soft_activation])
     

    feature_input = tf.keras.layers.Flatten()(feature_input)
    value_input=tf.keras.layers.Flatten()(value_input)
    
    for i in range(0,self.dense_layers):

      X_feature=tf.keras.layers.Dense(units =self.dense_nodes, activation = 'linear', name = "feature_CNN_dense"+str(i+1))(feature_input)

      feature_input=tf.keras.layers.Activation('relu')(X_feature)
      get_custom_objects().update({'custom_activation_soft': tf.keras.layers.Activation(soft_relu)})
      soft_activation=tf.keras.layers.Activation(soft_relu)(X_feature)

      X_value=tf.keras.layers.Dense(units =self.dense_nodes, activation = 'linear', name = "value_CNN_dense"+str(i+1))(value_input)
      value_input=tf.keras.layers.Multiply()([X_value,soft_activation])
      


      
    output_layer=tf.keras.layers.Dense(units =self.output_shape, activation = "softmax", name = "value_CNN_ouput")(value_input)

    self.model=tf.keras.Model(inputs=inputs,outputs=output_layer,name="decoupled_VCNN_model")

In [42]:
model=VCNN_decoupled_network(4,64,128,(3,3),2,256,32,10)

model.model.summary()


(None, 32, 32, 64)
(None, 32, 32, 64)
Model: "decoupled_VCNN_model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_15 (InputLayer)          [(None, 32, 32, 3)]  0           []                               
                                                                                                  
 feature_CNN_1 (Conv2D)         (None, 32, 32, 64)   1792        ['input_15[0][0]']               
                                                                                                  
 value_CNN_1 (Conv2D)           (None, 32, 32, 64)   1792        ['input_15[0][0]']               
                                                                                                  
 activation_52 (Activation)     (None, 32, 32, 64)   0           ['feature_CNN_1[0][0]']          
                                         

In [44]:
import tensorflow as tf
if __name__=="__main__":
  trainX,trainY,testX,testY=load_data(10)
  model=VCNN_decoupled_network(4,64,128,(3,3),2,256,32,10)
  model.model.summary()
  print(model.model.layers)
  model.model.compile(
  optimizer=tf.keras.optimizers.Adam(0.0003),  # Optimizer
  loss="categorical_crossentropy", metrics="accuracy")
  model.model.fit(trainX,trainY,epochs=5,batch_size=32,validation_data=(testX,testY))

(None, 32, 32, 64)
(None, 32, 32, 64)
Model: "decoupled_VCNN_model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_16 (InputLayer)          [(None, 32, 32, 3)]  0           []                               
                                                                                                  
 feature_CNN_1 (Conv2D)         (None, 32, 32, 64)   1792        ['input_16[0][0]']               
                                                                                                  
 value_CNN_1 (Conv2D)           (None, 32, 32, 64)   1792        ['input_16[0][0]']               
                                                                                                  
 activation_70 (Activation)     (None, 32, 32, 64)   0           ['feature_CNN_1[0][0]']          
                                         

KeyboardInterrupt: ignored

from keras.utils.generic_utils import get_custom_objects
def soft_relu(x):
    return tf.keras.backend.sigmoid(beta*x)